<a href="https://colab.research.google.com/github/wwoskie/Biostat_BI_2022/blob/biostat_project_1/biostat_project_1/biostat_project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Данные разбиты на несколько файлов. Нам надо как-то объединить наблюдения в единую таблицу. Пожалуйста, напишите пользовательскую функцию, благодаря которой мы сможем собрать все наши наблюдения в одну таблицу. Так как олимпиады будут проводиться регулярно, то функция должна объединять все файлы определенного расширения из заданной папки (тип расширения передается как аргумент функции). (5 баллов)

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
def table_merger(dir_name, file_extension, save_table = False, sep = ',', sheet = 0):
    
    def pd_read_csv_(path, sep, sheet):
        return pd.read_csv(path, sep = sep)
    def pf_read_excel_(path, sep, sheet):
        return pd.read_excel(path, sheet_name = sheet)
    
    reading_functions_dct = {
        pd_read_csv_: ['.csv', '.tsv', '.txt'], # я думаю, что все распространенные текстовые форматы сможет прочитать read_csv, только с соответствующими разделителями
        pf_read_excel_: ['.xlsx'] # для всего остального уже нет мастеркард(, но есть excel
    }
    
    reading_ext_dct = {} # выворачиваю словарь, чтобы обращаться по расширению
    for key, item in reading_functions_dct.items():
        for ext in item:
            reading_ext_dct[ext] = key
            
    
    files_to_read = [file for file in os.listdir(dir_name) if file.endswith(file_extension)]
    
    if len(files_to_read) > 0:
        df = reading_ext_dct[file_extension](path = os.path.join(dir_name, files_to_read[0]), sep = sep, sheet = sheet)
    else:
        df = None
        
    if len(files_to_read) > 1:
        for file in files_to_read[1:]:
            df = pd.concat([df, reading_ext_dct[file_extension](path = os.path.join(dir_name, file), sep = sep, sheet = sheet)])
            
    if save_table:
        df.to_csv(os.path.join(dir_name, f'merged_{files_to_read[0]}_{files_to_read[-1]}'), index=False)
    
    return df

In [3]:
df_ath = table_merger(dir_name = 'athlete_events', file_extension = '.csv')

2. Посмотрите, действительно ли все данные корректны? Если найдете, что что-то не так, то исправьте это, пожалуйста. Объясните, почему вы воспользовались именно этим подходом. Может быть у него есть альтернативы? (5 баллов)

In [4]:
df_ath

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,124517,William Van Dijck,M,23.0,185.0,65.0,Belgium,BEL,1984 Summer,1984.0,Summer,Los Angeles,Athletics,"Athletics Men's 3,000 metres Steeplechase",NaN
1,124517,William Van Dijck,M,27.0,185.0,65.0,Belgium,BEL,1988 Summer,1988.0,Summer,Seoul,Athletics,"Athletics Men's 3,000 metres Steeplechase",NaN
2,124517,William Van Dijck,M,31.0,185.0,65.0,Belgium,BEL,1992 Summer,1992.0,Summer,Barcelona,Athletics,"Athletics Men's 3,000 metres Steeplechase",NaN
3,124518,"Daniel ""Daan"" van Dijk",M,21.0,NaN,NaN,Netherlands,NED,1928 Summer,1928.0,Summer,Amsterdam,Cycling,"Cycling Men's Tandem Sprint, 2,000 metres",Gold
4,124519,"Everdina ""Edith"" van Dijk",F,35.0,176.0,66.0,Netherlands,NED,2008 Summer,2008.0,Summer,Beijing,Swimming,Swimming Women's 10 kilometres Open Water,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22831,12004,Peter Grant Blackburn,M,28.0,179.0,78.0,Australia,AUS,1996 Summer,1996.0,Summer,Atlanta,Badminton,Badminton Mixed Doubles,NaN
22832,12004,Peter Grant Blackburn,M,28.0,179.0,78.0,Australia,AUS,1996 Summer,1996.0,Summer,Atlanta,Badminton,Badminton Men's Doubles,NaN
22833,12004,Peter Grant Blackburn,M,32.0,179.0,78.0,Australia,AUS,2000 Summer,2000.0,Summer,Sydney,Badminton,Badminton Mixed Doubles,NaN
22834,12004,Peter Grant Blackburn,M,32.0,179.0,78.0,Australia,AUS,2000 Summer,2000.0,Summer,Sydney,Badminton,Badminton Men's Doubles,NaN


Так-с, ну в последней строке часть данных ушла в отпуск. Наверное, такие строки можно будет выявить по большому числу нанов, мне они не нравятся, я бы их выкинул

In [5]:
df_ath.apply(lambda x: x.count(), axis=1).value_counts()

14    177105
12     43633
15     30181
13     12459
11      7729
7          2
2          1
6          1
10         1
1          1
8          1
9          1
dtype: int64

Тут видно, что большая часть наблюдений содержит как минимум 11 записей в строке. Те, у кого 15 записей выиграли медали, они молодцы, те, у кого 14 записей тоже молодцы, так как съездили на олимпийские игры, но, скорее всего, медалей не привезли. Вопросы есть к тем, у кого записей меньше, почему...

In [6]:
df_ath[df_ath.apply(lambda x: x.count(), axis=1) < 11]

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
22409,113716,Pietro Spec,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22586,23433,Hadj Moussa Coulibaly,M,23.0,NaN,NaN,Mali,MLI,2004 Summe,NaN,NaN,NaN,NaN,NaN,NaN
22557,34727,Carlotta Ferlito,F,17.0,160.0,50.0,Italy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22642,68370,Helmut Lehmann,M,25.0,178.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22781,91137,Georgios Pantos,M,NaN,NaN,NaN,Athens-2,GRE,1906 Summer,1906.0,Summer,Athina,Footba,NaN,NaN
22463,124516,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22659,45919,Yuka Harada,F,28.0,170.0,60.0,Japan,JP,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22835,12005,Andrea Melissa Blackett,F,24.0,167.0,59.0,Barbados,BAR,2000 Su,NaN,NaN,NaN,NaN,NaN,NaN


Видим, что этих ребят уже не спасти, наверное, с ними придется расстаться(

In [7]:
df_ath = df_ath[df_ath.apply(lambda x: x.count(), axis=1) > 10]

Теперь попробуем пойти по колонкам и посмотреть, что тут у нас с заполненностью данными

In [8]:
for col in df_ath.columns:
    print(col, sum(pd.isnull(df_ath[col])))

ID 0
Name 0
Sex 0
Age 9473
Height 60168
Weight 62872
Team 0
NOC 0
Games 0
Year 0
Season 0
City 0
Sport 0
Event 0
Medal 231325


Больше всего пострадали медали (что логично, их мы трогать не будет), а также вес, рост и возраст. Возраст и рост можно попробовать воостановить по другим записям о спортсмене, вес тоже можно предположить и взять как среднее, например. Радует, что в айди нет пропущенных значений, но данные на всякий случай просмотреть, вдруг там что-то криво заполнено

In [9]:
no_age = set(df_ath[pd.isnull(df_ath['Age'])]['ID']) # только уникальные айди
len(set(no_age)) # без возраста вот столько вот уникальных спортсменов, попробуем его посчитать

6367

In [10]:
df_no_age = df_ath[df_ath['ID'].isin(no_age)] # в этот датафрейм должны попасть все спортсмены с айди без возраста. Вдруг для них есть другая запись
df_no_age

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
52,124544,Charles Van Doorselaer,M,NaN,NaN,NaN,Belgium,BEL,1920 Summer,1920.0,Summer,Antwerpen,Cycling,"Cycling Men's Team Pursuit, 4,000 metres",NaN
61,124550,R. Van Dorpe,M,NaN,NaN,NaN,Belgium,BEL,1932 Summer,1932.0,Summer,Los Angeles,Art Competitions,"Art Competitions Mixed Architecture, Unknown E...",NaN
79,124561,Herman Van Duyzen,M,NaN,NaN,NaN,Belgium,BEL,1924 Summer,1924.0,Summer,Paris,Wrestling,"Wrestling Men's Lightweight, Freestyle",NaN
182,124616,Louis Van Gheem,M,NaN,NaN,NaN,Belgium,BEL,1928 Summer,1928.0,Summer,Amsterdam,Water Polo,Water Polo Men's Water Polo,NaN
216,124627,Jean Van Guysse,M,NaN,NaN,NaN,Belgium,BEL,1908 Summer,1908.0,Summer,London,Gymnastics,Gymnastics Men's Individual All-Around,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22815,11999,Brian Reginald Blackburn,M,NaN,NaN,NaN,Australia,AUS,1956 Summer,1956.0,Summer,Melbourne,Gymnastics,Gymnastics Men's Horse Vault,NaN
22816,11999,Brian Reginald Blackburn,M,NaN,NaN,NaN,Australia,AUS,1956 Summer,1956.0,Summer,Melbourne,Gymnastics,Gymnastics Men's Parallel Bars,NaN
22817,11999,Brian Reginald Blackburn,M,NaN,NaN,NaN,Australia,AUS,1956 Summer,1956.0,Summer,Melbourne,Gymnastics,Gymnastics Men's Horizontal Bar,NaN
22818,11999,Brian Reginald Blackburn,M,NaN,NaN,NaN,Australia,AUS,1956 Summer,1956.0,Summer,Melbourne,Gymnastics,Gymnastics Men's Rings,NaN


In [11]:
col = 'Age'
print(col, sum(pd.isnull(df_no_age[col])))

Age 9473


К сожалению, это число совпадает с тем, что было получено ранее для возраста (ни для одного из ID нет дубликата с указанным возрастом), поэтому восстановить возраст не получится. Грустно(
Выбрасывать я их не буду, они могут пригодиться в других местах, ведь не везде придется считать что-то с возрастом
Проделаем аналогичные операции для роста

In [12]:
no_height = set(df_ath[pd.isnull(df_ath['Height'])]['ID']) # только уникальные айди
len(set(no_height)) # 

33914

In [13]:
df_no_height = df_ath[df_ath['ID'].isin(no_height)]
df_no_height

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
3,124518,"Daniel ""Daan"" van Dijk",M,21.0,NaN,NaN,Netherlands,NED,1928 Summer,1928.0,Summer,Amsterdam,Cycling,"Cycling Men's Tandem Sprint, 2,000 metres",Gold
10,124521,Hilbert Van Dijk,M,38.0,NaN,NaN,Australia,AUS,1956 Summer,1956.0,Summer,Melbourne,Fencing,"Fencing Men's epee, Team",NaN
14,124525,Johannes Wilhelmus Maria van Dijk,M,31.0,NaN,NaN,Minerva Amsterdam,NED,1900 Summer,1900.0,Summer,Paris,Rowing,Rowing Men's Coxed Eights,Bronze
16,124527,"Cornelis ""Nelis"" van Dijk",M,16.0,NaN,NaN,Netherlands,NED,1920 Summer,1920.0,Summer,Antwerpen,Boxing,Boxing Men's Flyweight,NaN
34,124534,"Gustaaf Lodewijk Gerhard ""Guus"" van Ditzhuyzen",M,19.0,NaN,NaN,Netherlands,NED,1928 Summer,1928.0,Summer,Amsterdam,Rowing,Rowing Men's Coxed Eights,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22818,11999,Brian Reginald Blackburn,M,NaN,NaN,NaN,Australia,AUS,1956 Summer,1956.0,Summer,Melbourne,Gymnastics,Gymnastics Men's Rings,NaN
22819,11999,Brian Reginald Blackburn,M,NaN,NaN,NaN,Australia,AUS,1956 Summer,1956.0,Summer,Melbourne,Gymnastics,Gymnastics Men's Pommelled Horse,NaN
22820,12000,Jerald Clifford Blackburn,M,20.0,NaN,NaN,Canada,CAN,1948 Summer,1948.0,Summer,London,Boxing,Boxing Men's Welterweight,NaN
22826,12002,Mackenzie Blackburn,M,21.0,NaN,NaN,Chinese Taipei,TPE,2014 Winter,2014.0,Winter,Sochi,Short Track Speed Skating,Short Track Speed Skating Men's 500 metres,NaN


In [14]:
col = 'Height'
print(col, sum(pd.isnull(df_no_height[col])))

Height 60168


Вот незадача! Тут тоже ничего не вышло, а ведь так хотелось... (ну либо я делаю что-то не так)

In [15]:
no_weight = set(df_ath[pd.isnull(df_ath['Weight'])]['ID']) # только уникальные айди
len(set(no_weight)) # 

34883

In [16]:
df_no_weight = df_ath[df_ath['ID'].isin(no_weight)]
df_no_weight

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
3,124518,"Daniel ""Daan"" van Dijk",M,21.0,NaN,NaN,Netherlands,NED,1928 Summer,1928.0,Summer,Amsterdam,Cycling,"Cycling Men's Tandem Sprint, 2,000 metres",Gold
10,124521,Hilbert Van Dijk,M,38.0,NaN,NaN,Australia,AUS,1956 Summer,1956.0,Summer,Melbourne,Fencing,"Fencing Men's epee, Team",NaN
14,124525,Johannes Wilhelmus Maria van Dijk,M,31.0,NaN,NaN,Minerva Amsterdam,NED,1900 Summer,1900.0,Summer,Paris,Rowing,Rowing Men's Coxed Eights,Bronze
16,124527,"Cornelis ""Nelis"" van Dijk",M,16.0,NaN,NaN,Netherlands,NED,1920 Summer,1920.0,Summer,Antwerpen,Boxing,Boxing Men's Flyweight,NaN
34,124534,"Gustaaf Lodewijk Gerhard ""Guus"" van Ditzhuyzen",M,19.0,NaN,NaN,Netherlands,NED,1928 Summer,1928.0,Summer,Amsterdam,Rowing,Rowing Men's Coxed Eights,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22818,11999,Brian Reginald Blackburn,M,NaN,NaN,NaN,Australia,AUS,1956 Summer,1956.0,Summer,Melbourne,Gymnastics,Gymnastics Men's Rings,NaN
22819,11999,Brian Reginald Blackburn,M,NaN,NaN,NaN,Australia,AUS,1956 Summer,1956.0,Summer,Melbourne,Gymnastics,Gymnastics Men's Pommelled Horse,NaN
22820,12000,Jerald Clifford Blackburn,M,20.0,NaN,NaN,Canada,CAN,1948 Summer,1948.0,Summer,London,Boxing,Boxing Men's Welterweight,NaN
22826,12002,Mackenzie Blackburn,M,21.0,NaN,NaN,Chinese Taipei,TPE,2014 Winter,2014.0,Winter,Sochi,Short Track Speed Skating,Short Track Speed Skating Men's 500 metres,NaN


In [17]:
col = 'Weight'
print(col, sum(pd.isnull(df_no_weight[col])))

Weight 62872


Чуда не случилось( Возможно, в некоторых олимпиадах спортсменов еще не было принято измерять или кто-то был в отпуске. Теперь посмотрим на валидность самих данных, вдруг тут кто-то что-то напутал при заполнении. Начнем с пола

In [18]:
df_ath['Sex'].value_counts() # тут появился третий пол

M    196585
F     74520
G         2
Name: Sex, dtype: int64

In [19]:
df_ath.loc[df_ath['Sex'] == 'G']

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
42,79609,Pavel Mike,G,22.0,182.0,79.0,Czechoslovakia,TCH,1972 Summer,1972.0,Summer,Munich,Handball,Handball Men's Handball,Silver
74,79630,Anatoly Mikhaylin,G,37.0,NaN,NaN,Russia,RUS,1996 Summer,1996.0,Summer,Atlanta,Sailing,Sailing Mixed Two Person Keelboat,NaN


Погуглил этих спортсменов. Скорее всего это просто ошибка при заполнении и они оба мужчины
https://wiki2.org/en/Pavel_Mike%C5%A1
https://en.wikipedia.org/wiki/Anatoly_Mikhaylin

In [20]:
df_ath.loc[df_ath['Sex'] == 'G', 'Sex'] = 'M' # исправим это в данных

In [21]:
df_ath['Sex'].value_counts()

M    196587
F     74520
Name: Sex, dtype: int64

Теперь посмотрим на возраст, в чате ходили слухи про детей и стариков, проверим правдивы ли они

In [22]:
df_ath['Age'].min(), df_ath['Age'].max()

(10.0, 240.0)

Ого, и правда, надо бы что-то с этим сделать. Для начала поймем, какие там вообще границы

In [23]:
all_ages = np.sort(df_ath['Age'][df_ath['Age'].notnull()].unique())
all_ages

array([ 10.,  11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,
        21.,  22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,
        32.,  33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,
        43.,  44.,  45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,
        54.,  55.,  56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,
        65.,  66.,  67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,
        76.,  77.,  80.,  81.,  84.,  88.,  96.,  97., 240.])

Скажем, 10 лет еще выглядят более-менее правдоподобно, но вот 240...

In [24]:
df_ath.loc[df_ath['Age'] == 240]

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
38,23459,Flicien Jules mile Courbet,M,240.0,NaN,NaN,Belgium,BEL,1912 Summer,1912.0,Summer,Stockholm,Swimming,Swimming Men's 200 metres Breaststroke,NaN


Видимо, тут лишний ноль и спортсмену было 24 года, потому что про него есть статья на википедии (род. 1888)
https://en.wikipedia.org/wiki/F%C3%A9licien_Courbet

In [25]:
df_ath.loc[df_ath['Age'] == 240, 'Age'] = 24 # исправим это в данных

In [26]:
all_ages = np.sort(df_ath['Age'][df_ath['Age'].notnull()].unique())
all_ages

array([10., 11., 12., 13., 14., 15., 16., 17., 18., 19., 20., 21., 22.,
       23., 24., 25., 26., 27., 28., 29., 30., 31., 32., 33., 34., 35.,
       36., 37., 38., 39., 40., 41., 42., 43., 44., 45., 46., 47., 48.,
       49., 50., 51., 52., 53., 54., 55., 56., 57., 58., 59., 60., 61.,
       62., 63., 64., 65., 66., 67., 68., 69., 70., 71., 72., 73., 74.,
       75., 76., 77., 80., 81., 84., 88., 96., 97.])

Взглянем на молодых спортсменов. Данные как будто выглядят правдоподобно, большинство представлено в тех видах спорта, где бывают молодые спортсмены.

In [27]:
df_ath[df_ath['Age'].isin(all_ages[:3])]

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
1078,125092,tienne Nol Henri Vandernotte,M,12.0,NaN,37.0,France,FRA,1936 Summer,1936.0,Summer,Berlin,Rowing,Rowing Men's Coxed Pairs,Bronze
1079,125092,tienne Nol Henri Vandernotte,M,12.0,NaN,37.0,France,FRA,1936 Summer,1936.0,Summer,Berlin,Rowing,Rowing Men's Coxed Fours,Bronze
2770,125944,Ines Vercesi,F,12.0,NaN,NaN,Italy,ITA,1928 Summer,1928.0,Summer,Amsterdam,Gymnastics,Gymnastics Women's Team All-Around,Silver
3505,126307,Liana Vicens,F,11.0,158.0,50.0,Puerto Rico,PUR,1968 Summer,1968.0,Summer,Mexico City,Swimming,Swimming Women's 100 metres Breaststroke,NaN
3506,126307,Liana Vicens,F,11.0,158.0,50.0,Puerto Rico,PUR,1968 Summer,1968.0,Summer,Mexico City,Swimming,Swimming Women's 200 metres Breaststroke,NaN
3507,126307,Liana Vicens,F,11.0,158.0,50.0,Puerto Rico,PUR,1968 Summer,1968.0,Summer,Mexico City,Swimming,Swimming Women's 200 metres Individual Medley,NaN
3508,126307,Liana Vicens,F,11.0,158.0,50.0,Puerto Rico,PUR,1968 Summer,1968.0,Summer,Mexico City,Swimming,Swimming Women's 4 x 100 metres Medley Relay,NaN
4937,127018,Yelena Germanovna Vodorezova (-Buyanova),F,12.0,161.0,47.0,Soviet Union,URS,1976 Winter,1976.0,Winter,Innsbruck,Figure Skating,Figure Skating Women's Singles,NaN
22168,113580,Inge Srensen (-Tabur),F,12.0,NaN,NaN,Denmark,DEN,1936 Summer,1936.0,Summer,Berlin,Swimming,Swimming Women's 200 metres Breaststroke,Bronze
20632,22411,Magdalena Cecilia Colledge,F,11.0,152.0,NaN,Great Britain,GBR,1932 Winter,1932.0,Winter,Lake Placid,Figure Skating,Figure Skating Women's Singles,NaN


Теперь посмотрим на возрастных. Ого, не знал, что были олимпийские игры по рисованию! В целом, данные тоже выглядят правдоподобно.

In [28]:
df_ath[df_ath['Age'].isin(all_ages[all_ages > 64])]

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
892,124990,"Cornelis Anthony ""Kees"" van Waning",M,66.0,NaN,NaN,Netherlands,NED,1928 Summer,1928.0,Summer,Amsterdam,Art Competitions,"Art Competitions Mixed Painting, Paintings",NaN
1989,125573,Daniel Vzquez Daz,M,66.0,NaN,NaN,Spain,ESP,1948 Summer,1948.0,Summer,London,Art Competitions,"Art Competitions Mixed Painting, Unknown Event",NaN
3775,126447,Johann Vierthaler,M,67.0,NaN,NaN,Germany,GER,1936 Summer,1936.0,Summer,Berlin,Art Competitions,"Art Competitions Mixed Sculpturing, Unknown Event",NaN
4574,126841,Konstantns Visotskis,M,68.0,NaN,NaN,Latvia,LAT,1932 Summer,1932.0,Summer,Los Angeles,Art Competitions,"Art Competitions Mixed Painting, Unknown Event",NaN
4575,126841,Konstantns Visotskis,M,68.0,NaN,NaN,Latvia,LAT,1932 Summer,1932.0,Summer,Los Angeles,Art Competitions,"Art Competitions Mixed Painting, Unknown Event",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21360,11270,"Lisa ""Louise"" Bianchini (Wadstrm-)",F,68.0,NaN,NaN,Sweden,SWE,1936 Summer,1936.0,Summer,Berlin,Art Competitions,"Art Competitions Mixed Painting, Paintings",NaN
21361,11270,"Lisa ""Louise"" Bianchini (Wadstrm-)",F,68.0,NaN,NaN,Sweden,SWE,1936 Summer,1936.0,Summer,Berlin,Art Competitions,"Art Competitions Mixed Painting, Paintings",NaN
21362,11270,"Lisa ""Louise"" Bianchini (Wadstrm-)",F,68.0,NaN,NaN,Sweden,SWE,1936 Summer,1936.0,Summer,Berlin,Art Competitions,"Art Competitions Mixed Painting, Paintings",NaN
21363,11270,"Lisa ""Louise"" Bianchini (Wadstrm-)",F,68.0,NaN,NaN,Sweden,SWE,1936 Summer,1936.0,Summer,Berlin,Art Competitions,"Art Competitions Mixed Painting, Paintings",NaN


In [29]:
df_ath[df_ath['Age'].isin(all_ages[-3:])]

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
8329,128719,John Quincy Adams Ward,M,97.0,NaN,NaN,United States,USA,1928 Summer,1928.0,Summer,Amsterdam,Art Competitions,"Art Competitions Mixed Sculpturing, Statues",NaN
15438,31173,Thomas Cowperthwait Eakins,M,88.0,NaN,NaN,United States,USA,1932 Summer,1932.0,Summer,Los Angeles,Art Competitions,"Art Competitions Mixed Painting, Unknown Event",NaN
15439,31173,Thomas Cowperthwait Eakins,M,88.0,NaN,NaN,United States,USA,1932 Summer,1932.0,Summer,Los Angeles,Art Competitions,"Art Competitions Mixed Painting, Unknown Event",NaN
15440,31173,Thomas Cowperthwait Eakins,M,88.0,NaN,NaN,United States,USA,1932 Summer,1932.0,Summer,Los Angeles,Art Competitions,"Art Competitions Mixed Painting, Unknown Event",NaN
7477,49663,Winslow Homer,M,96.0,NaN,NaN,United States,USA,1932 Summer,1932.0,Summer,Los Angeles,Art Competitions,"Art Competitions Mixed Painting, Unknown Event",NaN


Не менял ли какой-нибудь спортсмен возраст за одни игры?

In [56]:
df_n_ages = df_ath.groupby(['ID', 'Games']).agg(a_unique = ('Age', 'unique'))
df_n_ages['n_ages'] = df_n_ages.a_unique.map(len)
df_n_ages.loc[df_n_ages['n_ages'] > 1]

,,a_unique,n_ages
ID,Games,,


С полом и возрастом разобрались, теперь взглянем на рост

In [30]:
all_height = np.sort(df_ath['Height'][df_ath['Height'].notnull()].unique())
all_height

array([127., 128., 130., 131., 132., 133., 135., 136., 137., 138., 139.,
       140., 141., 142., 143., 144., 145., 146., 147., 148., 149., 150.,
       151., 152., 153., 154., 155., 156., 157., 158., 159., 160., 161.,
       162., 163., 164., 165., 166., 167., 168., 169., 170., 171., 172.,
       173., 174., 175., 176., 177., 178., 179., 180., 181., 182., 183.,
       184., 185., 186., 187., 188., 189., 190., 191., 192., 193., 194.,
       195., 196., 197., 198., 199., 200., 201., 202., 203., 204., 205.,
       206., 207., 208., 209., 210., 211., 212., 213., 214., 215., 216.,
       217., 218., 219., 220., 221., 223., 226., 340.])

Рост в почти 3.5 метра выглядит не ок, взглянем

In [31]:
df_ath.loc[df_ath['Height'] == 340]

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
227,23549,Kirsty Leigh Coventry (-Seward),F,28.0,340.0,64.0,Zimbabwe,ZIM,2012 Summer,2012.0,Summer,London,Swimming,Swimming Women's 200 metres Individual Medley,NaN


А есть ли еще люди, которые меняли рост?

In [32]:
df_n_heights = df_ath.groupby(['ID']).agg(h_unique = ('Height', 'unique'))
df_n_heights['n_heights'] = df_n_heights.h_unique.map(len)
df_n_heights.loc[df_n_heights['n_heights'] > 1]

,h_unique,n_heights
ID,,
23549,"[176.0, 340.0]",2


Нету, и это странно!

In [33]:
df_ath.loc[df_ath['ID'] == 23549]

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
214,23549,Kirsty Leigh Coventry (-Seward),F,16.0,176.0,64.0,Zimbabwe,ZIM,2000 Summer,2000.0,Summer,Sydney,Swimming,Swimming Women's 50 metres Freestyle,NaN
215,23549,Kirsty Leigh Coventry (-Seward),F,16.0,176.0,64.0,Zimbabwe,ZIM,2000 Summer,2000.0,Summer,Sydney,Swimming,Swimming Women's 100 metres Freestyle,NaN
216,23549,Kirsty Leigh Coventry (-Seward),F,16.0,176.0,64.0,Zimbabwe,ZIM,2000 Summer,2000.0,Summer,Sydney,Swimming,Swimming Women's 100 metres Backstroke,NaN
217,23549,Kirsty Leigh Coventry (-Seward),F,16.0,176.0,64.0,Zimbabwe,ZIM,2000 Summer,2000.0,Summer,Sydney,Swimming,Swimming Women's 200 metres Individual Medley,NaN
218,23549,Kirsty Leigh Coventry (-Seward),F,20.0,176.0,64.0,Zimbabwe,ZIM,2004 Summer,2004.0,Summer,Athina,Swimming,Swimming Women's 100 metres Backstroke,Silver
219,23549,Kirsty Leigh Coventry (-Seward),F,20.0,176.0,64.0,Zimbabwe,ZIM,2004 Summer,2004.0,Summer,Athina,Swimming,Swimming Women's 200 metres Backstroke,Gold
220,23549,Kirsty Leigh Coventry (-Seward),F,20.0,176.0,64.0,Zimbabwe,ZIM,2004 Summer,2004.0,Summer,Athina,Swimming,Swimming Women's 200 metres Individual Medley,Bronze
221,23549,Kirsty Leigh Coventry (-Seward),F,24.0,176.0,64.0,Zimbabwe,ZIM,2008 Summer,2008.0,Summer,Beijing,Swimming,Swimming Women's 100 metres Backstroke,Silver
222,23549,Kirsty Leigh Coventry (-Seward),F,24.0,176.0,64.0,Zimbabwe,ZIM,2008 Summer,2008.0,Summer,Beijing,Swimming,Swimming Women's 200 metres Backstroke,Gold
223,23549,Kirsty Leigh Coventry (-Seward),F,24.0,176.0,64.0,Zimbabwe,ZIM,2008 Summer,2008.0,Summer,Beijing,Swimming,Swimming Women's 200 metres Individual Medley,Silver


Завидная стабильность в весе! Про рост такого не скажешь, поэтому заменим

In [34]:
df_ath.loc[df_ath['Height'] == 340, 'Height'] = 176

Взглянем на вес

In [36]:
all_weights = np.sort(df_ath['Weight'][df_ath['Weight'].notnull()].unique())
all_weights

array([ 25.        ,  28.        ,  30.        ,  31.        ,
        32.        ,  33.        ,  34.        ,  35.        ,
        36.        ,  37.        ,  38.        ,  39.        ,
        40.        ,  41.        ,  42.        ,  43.        ,
        44.        ,  45.        ,  46.        ,  47.        ,
        48.        ,  48.5       ,  49.        ,  49.5       ,
        50.        ,  51.        ,  51.5       ,  52.        ,
        52.5       ,  53.        ,  53.5       ,  54.        ,
        54.5       ,  55.        ,  55.5       ,  56.        ,
        56.5       ,  57.        ,  57.5       ,  58.        ,
        58.5       ,  59.        ,  59.5       ,  60.        ,
        60.5       ,  61.        ,  61.5       ,  62.        ,
        62.5       ,  63.        ,  63.5       ,  64.        ,
        64.5       ,  65.        ,  65.5       ,  66.        ,
        66.5       ,  67.        ,  67.5       ,  68.        ,
        68.5       ,  69.        ,  69.5       ,  70.  

In [37]:
df_ath[df_ath['Weight'].isin(all_weights[-5:])]

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
1616,103159,Aytami Ruano Vega,M,27.0,200.0,198.0,Spain,ESP,2004 Summer,2004.0,Summer,Athina,Judo,Judo Men's Heavyweight,NaN
2501,103580,Valentyn Rusliakov,M,28.0,187.0,180.0,Ukraine,UKR,2000 Summer,2000.0,Summer,Sydney,Judo,Judo Men's Heavyweight,NaN
319,12177,"Ricardo Blas, Jr.",M,21.0,183.0,214.0,Guam,GUM,2008 Summer,2008.0,Summer,Beijing,Judo,Judo Men's Heavyweight,NaN
320,12177,"Ricardo Blas, Jr.",M,25.0,183.0,214.0,Guam,GUM,2012 Summer,2012.0,Summer,London,Judo,Judo Men's Heavyweight,NaN
10778,118869,"Christopher J. ""Chris"" Taylor",M,22.0,196.0,182.0,United States,USA,1972 Summer,1972.0,Summer,Munich,Wrestling,"Wrestling Men's Super-Heavyweight, Greco-Roman",NaN
10779,118869,"Christopher J. ""Chris"" Taylor",M,22.0,196.0,182.0,United States,USA,1972 Summer,1972.0,Summer,Munich,Wrestling,"Wrestling Men's Super-Heavyweight, Freestyle",Bronze
7050,38075,Marek Galiski,M,29.0,200.0,190.0,Poland,POL,1980 Summer,1980.0,Summer,Moskva,Wrestling,"Wrestling Men's Super-Heavyweight, Greco-Roman",NaN


Самый большой человек тут и правда очень большой... https://www.google.com/search?q=Ricardo+Blas,+Jr.&sxsrf=ALiCzsZLiz3B-dvisUdZQCG8ZQhtrlN_5g:1669258526746&source=lnms&tbm=isch&sa=X&ved=2ahUKEwi4_JjM6MX7AhX-i_0HHfMfB_oQ_AUoAXoECAMQAw&biw=1920&bih=1007&dpr=1


In [39]:
df_ath[df_ath['Weight'].isin(all_weights[:2])]

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
8111,128605,Wang Xin (Ruoxue-),F,15.0,137.0,28.0,China,CHN,2008 Summer,2008.0,Summer,Beijing,Diving,Diving Women's Platform,Bronze
8112,128605,Wang Xin (Ruoxue-),F,15.0,137.0,28.0,China,CHN,2008 Summer,2008.0,Summer,Beijing,Diving,Diving Women's Synchronized Platform,Gold
15814,132383,Kana Yamawaki,F,16.0,141.0,28.0,Japan,JPN,2000 Summer,2000.0,Summer,Sydney,Gymnastics,Gymnastics Women's Individual All-Around,NaN
15815,132383,Kana Yamawaki,F,16.0,141.0,28.0,Japan,JPN,2000 Summer,2000.0,Summer,Sydney,Gymnastics,Gymnastics Women's Floor Exercise,NaN
15816,132383,Kana Yamawaki,F,16.0,141.0,28.0,Japan,JPN,2000 Summer,2000.0,Summer,Sydney,Gymnastics,Gymnastics Women's Horse Vault,NaN
15817,132383,Kana Yamawaki,F,16.0,141.0,28.0,Japan,JPN,2000 Summer,2000.0,Summer,Sydney,Gymnastics,Gymnastics Women's Uneven Bars,NaN
15818,132383,Kana Yamawaki,F,16.0,141.0,28.0,Japan,JPN,2000 Summer,2000.0,Summer,Sydney,Gymnastics,Gymnastics Women's Balance Beam,NaN
19861,134370,"Albert Ferdinand ""Al"" Zerhusen",M,24.0,183.0,28.0,United States,USA,1956 Summer,1956.0,Summer,Melbourne,Football,Football Men's Football,NaN
18013,21049,Choi Myong-Hui,F,14.0,135.0,25.0,North Korea,PRK,1980 Summer,1980.0,Summer,Moskva,Gymnastics,Gymnastics Women's Individual All-Around,NaN
18014,21049,Choi Myong-Hui,F,14.0,135.0,25.0,North Korea,PRK,1980 Summer,1980.0,Summer,Moskva,Gymnastics,Gymnastics Women's Team All-Around,NaN


С ними все тоже наверное окк. Проверим возможные ошибки при заполнении веса, посмотрев число уникальных значений внутри одной олимпиады

In [44]:
df_n_weights = df_ath.groupby(['ID', 'Games']).agg(w_unique = ('Weight', 'unique'))
df_n_weights['n_weights'] = df_n_weights.w_unique.map(len)
df_n_weights.loc[df_n_weights['n_weights'] > 1]

,,w_unique,n_weights
ID,Games,,
23466,2004 Summer,"[65.0, 103.0]",2


In [45]:
df_ath.loc[df_ath['ID'] == 23466]

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
55,23466,Didier Courrges,M,44.0,171.0,65.0,France,FRA,2004 Summer,2004.0,Summer,Athina,Equestrianism,"Equestrianism Mixed Three-Day Event, Individual",NaN
56,23466,Didier Courrges,M,44.0,171.0,103.0,France,FRA,2004 Summer,2004.0,Summer,Athina,Equestrianism,"Equestrianism Mixed Three-Day Event, Team",Gold


Так, а кому в итоге верить? Попробуем поверить медиане (и тому факту, что он занимается конным спортом https://en.wikipedia.org/wiki/Didier_Courr%C3%A8ges, а там чем легче человек тем лучше) 

In [54]:
df_ath.query('Sex == "M" and Sport == "Equestrianism"')['Weight'].median()

70.0

In [55]:
df_ath.loc[df_ath['ID'] == 23466, 'Weight'] = 65

Взглянем на олимпийские комитеты

In [57]:
all_noc = np.sort(df_ath['NOC'][df_ath['NOC'].notnull()].unique())
all_noc

array(['AFG', 'AHO', 'ALB', 'ALG', 'AND', 'ANG', 'ANT', 'ANZ', 'ARG',
       'ARM', 'ARU', 'ASA', 'AUS', 'AUT', 'AZE', 'BAH', 'BAN', 'BAR',
       'BDI', 'BEL', 'BEN', 'BER', 'BHU', 'BIH', 'BIZ', 'BLR', 'BOH',
       'BOL', 'BOT', 'BRA', 'BRN', 'BRU', 'BUL', 'BUR', 'CAF', 'CAM',
       'CAN', 'CAY', 'CGO', 'CHA', 'CHI', 'CHN', 'CIV', 'CMR', 'COD',
       'COK', 'COL', 'COM', 'CPV', 'CRC', 'CRO', 'CRT', 'CUB', 'CYP',
       'CZE', 'DEN', 'DJI', 'DMA', 'DOM', 'ECU', 'EGY', 'ERI', 'ESA',
       'ESP', 'EST', 'ETH', 'EUN', 'FIJ', 'FIN', 'FRA', 'FRG', 'FSM',
       'GAB', 'GAM', 'GBR', 'GBS', 'GDR', 'GEO', 'GEQ', 'GER', 'GHA',
       'GRE', 'GRN', 'GUA', 'GUI', 'GUM', 'GUY', 'HAI', 'HKG', 'HON',
       'HUN', 'INA', 'IND', 'IOA', 'IRI', 'IRL', 'IRQ', 'ISL', 'ISR',
       'ISV', 'ITA', 'IVB', 'JAM', 'JOR', 'JPN', 'KAZ', 'KEN', 'KGZ',
       'KIR', 'KOR', 'KOS', 'KSA', 'KUW', 'LAO', 'LAT', 'LBA', 'LBR',
       'LCA', 'LES', 'LIB', 'LIE', 'LTU', 'LUX', 'MAD', 'MAL', 'MAR',
       'MAS', 'MAW',

Вроде с ними все ок. Смотрим на игры

In [77]:
all_games = np.sort(df_ath['Games'][df_ath['Games'].notnull()].unique())
all_games

array(['1896 Summer', '1900 Summer', '1904 Summer', '1906 Summer',
       '1908 Summer', '1912 Summer', '1920 Summer', '1924 Summer',
       '1924 Winter', '1928 Summer', '1928 Winter', '1932 Summer',
       '1932 Winter', '1936 Summer', '1936 Winter', '1948 Summer',
       '1948 Winter', '1952 Summer', '1952 Winter', '1956 Summer',
       '1956 Winter', '1960 Summer', '1960 Winter', '1964 Summer',
       '1964 Winter', '1968 Summer', '1968 Winter', '1972 Summer',
       '1972 Winter', '1976 Summer', '1976 Winter', '1980 Summer',
       '1980 Winter', '1984 Summer', '1984 Winter', '1988 Summer',
       '1988 Winter', '1992 Summer', '1992 Winter', '1994 Winter',
       '1996 Summer', '1998 Winter', '2000 Summer', '2002 Winter',
       '2004 Summer', '2006 Winter', '2008 Summer', '2010 Winter',
       '2012 Summer', '2014 Winter', '2016 Summer'], dtype=object)

С ними все ок. Наверное, есть смысл в отдельную колонку вынести сезон игр

In [62]:
df_ath[['Season_year', 'Season']] = df_ath.Games.str.split(' ',expand=True)

/tmp/ipykernel_17637/1980172255.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ath[['Season_year', 'Season']] = df_ath.Games.str.split(' ',expand=True)


In [63]:
df_ath

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,Season_year
0,124517,William Van Dijck,M,23.0,185.0,65.0,Belgium,BEL,1984 Summer,1984.0,Summer,Los Angeles,Athletics,"Athletics Men's 3,000 metres Steeplechase",NaN,1984
1,124517,William Van Dijck,M,27.0,185.0,65.0,Belgium,BEL,1988 Summer,1988.0,Summer,Seoul,Athletics,"Athletics Men's 3,000 metres Steeplechase",NaN,1988
2,124517,William Van Dijck,M,31.0,185.0,65.0,Belgium,BEL,1992 Summer,1992.0,Summer,Barcelona,Athletics,"Athletics Men's 3,000 metres Steeplechase",NaN,1992
3,124518,"Daniel ""Daan"" van Dijk",M,21.0,NaN,NaN,Netherlands,NED,1928 Summer,1928.0,Summer,Amsterdam,Cycling,"Cycling Men's Tandem Sprint, 2,000 metres",Gold,1928
4,124519,"Everdina ""Edith"" van Dijk",F,35.0,176.0,66.0,Netherlands,NED,2008 Summer,2008.0,Summer,Beijing,Swimming,Swimming Women's 10 kilometres Open Water,NaN,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22830,12003,Michael James Blackburn,M,34.0,178.0,78.0,Australia,AUS,2004 Summer,2004.0,Summer,Athina,Sailing,Sailing Mixed One Person Dinghy,NaN,2004
22831,12004,Peter Grant Blackburn,M,28.0,179.0,78.0,Australia,AUS,1996 Summer,1996.0,Summer,Atlanta,Badminton,Badminton Mixed Doubles,NaN,1996
22832,12004,Peter Grant Blackburn,M,28.0,179.0,78.0,Australia,AUS,1996 Summer,1996.0,Summer,Atlanta,Badminton,Badminton Men's Doubles,NaN,1996
22833,12004,Peter Grant Blackburn,M,32.0,179.0,78.0,Australia,AUS,2000 Summer,2000.0,Summer,Sydney,Badminton,Badminton Mixed Doubles,NaN,2000


In [69]:
sum(df_ath['Year'] != pd.to_numeric(df_ath['Season_year']))

0

In [73]:
sum(df_ath['Year'].isnull()), sum(df_ath['Season_year'].isnull())

(0, 0)

Значит, ошибок при заполнении нет

In [74]:
df_ath.drop(columns=['Season_year'], inplace = True)

/tmp/ipykernel_17637/547514310.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ath.drop(columns=['Season_year'], inplace = True)


In [75]:
df_ath

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,124517,William Van Dijck,M,23.0,185.0,65.0,Belgium,BEL,1984 Summer,1984.0,Summer,Los Angeles,Athletics,"Athletics Men's 3,000 metres Steeplechase",NaN
1,124517,William Van Dijck,M,27.0,185.0,65.0,Belgium,BEL,1988 Summer,1988.0,Summer,Seoul,Athletics,"Athletics Men's 3,000 metres Steeplechase",NaN
2,124517,William Van Dijck,M,31.0,185.0,65.0,Belgium,BEL,1992 Summer,1992.0,Summer,Barcelona,Athletics,"Athletics Men's 3,000 metres Steeplechase",NaN
3,124518,"Daniel ""Daan"" van Dijk",M,21.0,NaN,NaN,Netherlands,NED,1928 Summer,1928.0,Summer,Amsterdam,Cycling,"Cycling Men's Tandem Sprint, 2,000 metres",Gold
4,124519,"Everdina ""Edith"" van Dijk",F,35.0,176.0,66.0,Netherlands,NED,2008 Summer,2008.0,Summer,Beijing,Swimming,Swimming Women's 10 kilometres Open Water,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22830,12003,Michael James Blackburn,M,34.0,178.0,78.0,Australia,AUS,2004 Summer,2004.0,Summer,Athina,Sailing,Sailing Mixed One Person Dinghy,NaN
22831,12004,Peter Grant Blackburn,M,28.0,179.0,78.0,Australia,AUS,1996 Summer,1996.0,Summer,Atlanta,Badminton,Badminton Mixed Doubles,NaN
22832,12004,Peter Grant Blackburn,M,28.0,179.0,78.0,Australia,AUS,1996 Summer,1996.0,Summer,Atlanta,Badminton,Badminton Men's Doubles,NaN
22833,12004,Peter Grant Blackburn,M,32.0,179.0,78.0,Australia,AUS,2000 Summer,2000.0,Summer,Sydney,Badminton,Badminton Mixed Doubles,NaN


In [79]:
all_cities = np.sort(df_ath['City'][df_ath['City'].notnull()].unique())
all_cities

array(['Albertville', 'Amsterdam', 'Antwerpen', 'Athina', 'Atlanta',
       'Barcelona', 'Beijing', 'Berlin', 'Calgary', 'Chamonix',
       "Cortina d'Ampezzo", 'Garmisch-Partenkirchen', 'Grenoble',
       'Helsinki', 'Innsbruck', 'Lake Placid', 'Lillehammer', 'London',
       'Los Angeles', 'Melbourne', 'Mexico City', 'Montreal', 'Moskva',
       'Munich', 'Nagano', 'Oslo', 'Paris', 'Rio de Janeiro', 'Roma',
       'Salt Lake City', 'Sankt Moritz', 'Sapporo', 'Sarajevo', 'Seoul',
       'Sochi', 'Squaw Valley', 'St. Louis', 'Stockholm', 'Sydney',
       'Tokyo', 'Torino', 'Vancouver'], dtype=object)

С городами все вроде тоже ок

In [81]:
all_events = np.sort(df_ath['Event'][df_ath['Event'].notnull()].unique())
#all_events # слишком большой список, но вроде бы с событиями все тоже ок

In [82]:
df_ath['Medal'].value_counts() # с заполнением медалей тоже ок

Gold      13372
Bronze    13295
Silver    13115
Name: Medal, dtype: int64

Часть с "взглянуть на данные" наверное самая ручная, долгая и неприятная, если честно, но неск ошибок удалось исправить, чему нельзя не радоваться. Надо бы объяснить, почему я не стал заполнять NaNы: в принципе их можно дропнуть на любом этапе анализа так сказать *in situ*, а сместить распределение, заполнив какой-нибудь медианой или средним, например, мне бы не хотелось, т.к. я слышал про тенденции со "сдвигом" параметров спортсменов со временем к наиболее оптимальным, а заполнить по группам (например, взяв медиану по соседним олимпийским играм) кажется слишком сложным. Впрочем, мое мнение все еще может поменяться 

3. Выясните возраст самых молодых спортсменов обоих полов на Олимпиаде 1992 года. (2 балла)

In [95]:
df_ath.loc[df_ath['Year'] == 1992].groupby('Sex')['Age'].agg('min').reset_index()

,Sex,Age
0,F,12.0
1,M,11.0


Рассчитайте среднее значение и стандартное отклонение переменной Height для
спортсменов каждого пола. (2 балла)

In [96]:
df_ath.groupby('Sex')['Height'].agg(['mean', 'std']).reset_index()

,Sex,mean,std
0,F,167.839824,8.778602
1,M,178.858620,9.360377


5. Рассчитайте среднее значение и стандартное отклонение переменной Height у теннисисток (sex = F) на Олимпиаде 2000 года. Округлите ответ до первого знака после точки. (2 балла)

In [102]:
df_ath.query('Sex == "F" and Sport == "Tennis" and Year == 2000').agg({'Height': ['mean', 'std']}).round(1)

,Height
mean,171.8
std,6.5


6. В каком виде спорта участвовал самый тяжеловесный атлет на Олимпиаде 2006 года?

In [106]:
df_ath.query('Sport == "Athletics" and Year == 2006')

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal


Тут пусто, т.к. в 2006 году были зимние игры и атлетики не было. Я бы ответил так. Но на всякий случай поищем самого тяжелого спортсмена в принципе (я не понимаю атлет == спортсмен или нет)

In [113]:
df_ath.query('Year == 2006').loc[df_ath.query('Year == 2006')['Weight'] == df_ath.query('Year == 2006')['Weight'].max()]

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
8102,4476,Patrick R. Antaki,M,41.0,185.0,127.0,Lebanon,LIB,2006 Winter,2006.0,Winter,Torino,Skeleton,Skeleton Men's Skeleton,NaN


Ну ок, в скелетоне, допустим

7. Какое количество золотых медалей было получено женщинами с 1980 по 2010 года? (2 балла)

In [116]:
year_range = np.arange(1980, 2011)

In [131]:
year_range

array([1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990,
       1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001,
       2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010])

In [135]:
df_ath.query('Sex == "F" and Medal == "Gold"')['Year'].isin(year_range).sum()

2249

8. Сколько раз спортсмен John Aalberg участвовал в Олимпийских играх в разные годы? (2 балла)

In [142]:
df_ath.query('Name == "John Aalberg"').groupby('Year').ngroups

2

9. Определите наименее и наиболее представленные (по числу участников) возрастные группы спортсменов на Олимпийских играх 2008 года. Возможные возрастные группы: [15-25), [25-35), [35-45), [45-55]. (4 балла)

Окей, допустим, мы знаем, что есть спортсмены и моложе и старше

In [148]:
age_groups_dct = {
    '<15' : np.arange(0, 15), # creating ranges for age groups
    '15-25' : np.arange(15, 25),
    '25-35' : np.arange(25, 35),
    '35-45' : np.arange(35, 45),
    '45-55' : np.arange(45, 55),
    '>55' : np.arange(55, 10000)
    }
    

In [150]:
df_ath_age_groups = df_ath # creating dummy df to add age groups

for age_group in age_groups_dct:
    df_ath_age_groups.loc[df_ath_age_groups['Age'].isin(age_groups_dct[age_group]), 'Age_group'] = age_group

/tmp/ipykernel_17637/3394831107.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ath_age_groups.loc[df_ath_age_groups['Age'].isin(age_groups_dct[age_group]), 'Age_group'] = age_group


In [155]:
df_ath_age_groups.query('Year == 2008').groupby('Age_group')['ID'].count()

Age_group
15-25    6294
25-35    6367
35-45     790
45-55     116
<15        17
>55        16
Name: ID, dtype: int64

We can see that most represented group is 25-25, the least represented is >55

10. Насколько изменилось число видов спорта на Олимпиаде 2002 года по сравнению с Олимпийскими играми 1994 года? (2 балла)

In [272]:
df_n_sports = df_ath.query('Year == 2002 or Year == 1994').groupby('Year')['Sport'].count().reset_index()
df_n_sports

,Year,Sport
0,1994.0,3160
1,2002.0,4109


In [273]:
df_n_sports['Sport'][1] - df_n_sports['Sport'][0]

949

Almost 1000 more, wow

11. Выведите для зимней и летней олимпиады отдельно топ 3 стран по каждой из типов медалей (2 балла)

In [286]:
df_ath.groupby(['Team', 'Season','Medal'])['Medal'].agg(N_medals = 'count').reset_index().set_index('Team').groupby(['Season', 'Medal'])['N_medals'].nlargest(3).to_frame()

N_medals
Season Medal  Team                   
Summer Bronze United States      1112
              Germany             610
              Soviet Union        596
       Gold   United States      2333
              Soviet Union        830
              Germany             564
       Silver United States      1241
              Soviet Union        635
              Great Britain       576
Winter Bronze Finland             215
              Sweden              177
              Norway              127
       Gold   Canada              289
              Soviet Union        228
              Norway              151
       Silver United States       271
              Canada              189
              Norway              165

12. Создайте новую переменную Height_z_scores и сохраните в нее значения переменной Height после ее стандартизации. (2 балла)

In [242]:
import scipy
from scipy import stats

In [247]:
df_ath['Height_z_scores'] = stats.zscore(df_ath['Height'], nan_policy='omit')

/tmp/ipykernel_17637/1093548832.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ath['Height_z_scores'] = stats.zscore(df_ath['Height'], nan_policy='omit')


In [248]:
df_ath

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,Age_group,Height_z_scores
0,124517,William Van Dijck,M,23.0,185.0,65.0,Belgium,BEL,1984 Summer,1984.0,Summer,Los Angeles,Athletics,"Athletics Men's 3,000 metres Steeplechase",NaN,15-25,0.918466
1,124517,William Van Dijck,M,27.0,185.0,65.0,Belgium,BEL,1988 Summer,1988.0,Summer,Seoul,Athletics,"Athletics Men's 3,000 metres Steeplechase",NaN,25-35,0.918466
2,124517,William Van Dijck,M,31.0,185.0,65.0,Belgium,BEL,1992 Summer,1992.0,Summer,Barcelona,Athletics,"Athletics Men's 3,000 metres Steeplechase",NaN,25-35,0.918466
3,124518,"Daniel ""Daan"" van Dijk",M,21.0,NaN,NaN,Netherlands,NED,1928 Summer,1928.0,Summer,Amsterdam,Cycling,"Cycling Men's Tandem Sprint, 2,000 metres",Gold,15-25,NaN
4,124519,"Everdina ""Edith"" van Dijk",F,35.0,176.0,66.0,Netherlands,NED,2008 Summer,2008.0,Summer,Beijing,Swimming,Swimming Women's 10 kilometres Open Water,NaN,35-45,0.062831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22830,12003,Michael James Blackburn,M,34.0,178.0,78.0,Australia,AUS,2004 Summer,2004.0,Summer,Athina,Sailing,Sailing Mixed One Person Dinghy,NaN,25-35,0.252972
22831,12004,Peter Grant Blackburn,M,28.0,179.0,78.0,Australia,AUS,1996 Summer,1996.0,Summer,Atlanta,Badminton,Badminton Mixed Doubles,NaN,25-35,0.348043
22832,12004,Peter Grant Blackburn,M,28.0,179.0,78.0,Australia,AUS,1996 Summer,1996.0,Summer,Atlanta,Badminton,Badminton Men's Doubles,NaN,25-35,0.348043
22833,12004,Peter Grant Blackburn,M,32.0,179.0,78.0,Australia,AUS,2000 Summer,2000.0,Summer,Sydney,Badminton,Badminton Mixed Doubles,NaN,25-35,0.348043


13. Дополнительно: Создайте новую переменную Height_min_max_scaled и сохраните в нее значения переменной Height после применения к ней min-max нормализации (нужно будет разобраться, как она работает). (2 балла)

In [250]:
from sklearn.preprocessing import MinMaxScaler

In [262]:
df_ath['Height_min_max_scaled'] = MinMaxScaler().fit_transform(df_ath['Height'].to_numpy().reshape(-1, 1))

/tmp/ipykernel_17637/482566265.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ath['Height_min_max_scaled'] = MinMaxScaler().fit_transform(df_ath['Height'].to_numpy().reshape(-1, 1))


In [263]:
df_ath

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,Age_group,Height_z_scores,Height_min_max_scaled
0,124517,William Van Dijck,M,23.0,185.0,65.0,Belgium,BEL,1984 Summer,1984.0,Summer,Los Angeles,Athletics,"Athletics Men's 3,000 metres Steeplechase",NaN,15-25,0.918466,0.585859
1,124517,William Van Dijck,M,27.0,185.0,65.0,Belgium,BEL,1988 Summer,1988.0,Summer,Seoul,Athletics,"Athletics Men's 3,000 metres Steeplechase",NaN,25-35,0.918466,0.585859
2,124517,William Van Dijck,M,31.0,185.0,65.0,Belgium,BEL,1992 Summer,1992.0,Summer,Barcelona,Athletics,"Athletics Men's 3,000 metres Steeplechase",NaN,25-35,0.918466,0.585859
3,124518,"Daniel ""Daan"" van Dijk",M,21.0,NaN,NaN,Netherlands,NED,1928 Summer,1928.0,Summer,Amsterdam,Cycling,"Cycling Men's Tandem Sprint, 2,000 metres",Gold,15-25,NaN,NaN
4,124519,"Everdina ""Edith"" van Dijk",F,35.0,176.0,66.0,Netherlands,NED,2008 Summer,2008.0,Summer,Beijing,Swimming,Swimming Women's 10 kilometres Open Water,NaN,35-45,0.062831,0.494949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22830,12003,Michael James Blackburn,M,34.0,178.0,78.0,Australia,AUS,2004 Summer,2004.0,Summer,Athina,Sailing,Sailing Mixed One Person Dinghy,NaN,25-35,0.252972,0.515152
22831,12004,Peter Grant Blackburn,M,28.0,179.0,78.0,Australia,AUS,1996 Summer,1996.0,Summer,Atlanta,Badminton,Badminton Mixed Doubles,NaN,25-35,0.348043,0.525253
22832,12004,Peter Grant Blackburn,M,28.0,179.0,78.0,Australia,AUS,1996 Summer,1996.0,Summer,Atlanta,Badminton,Badminton Men's Doubles,NaN,25-35,0.348043,0.525253
22833,12004,Peter Grant Blackburn,M,32.0,179.0,78.0,Australia,AUS,2000 Summer,2000.0,Summer,Sydney,Badminton,Badminton Mixed Doubles,NaN,25-35,0.348043,0.525253


14. Сравните рост, вес и возраст мужчин и женщин, участвовавших в зимних олимпийских играх. Пожалуйста, оформите результаты так, чтобы мы сразу могли использовать их для статьи. (5 баллов)

In [305]:
df_ath_winter = df_ath.query('Season == "Winter"').groupby(['Sex', 'ID'])['Name'].agg('count')

In [306]:
df_ath_winter

Sex  ID    
F    5         6
     14        1
     26        1
     126       1
     422       5
              ..
M    135559    7
     135564    1
     135569    1
     135570    2
     135571    2
Name: Name, Length: 18959, dtype: int64